In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [2]:
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad  # 默认值是None

x.grad 默认情况下为 None 是因为梯度只有在调用 backward() 之后才会被计算并存储。之前在创建张量 x 时，虽然设置了 requires_grad=True，但还没有进行任何梯度计算。所以，在 y.backward() 之前，x.grad 为空。只有在进行反向传播（backward()）之后，x.grad 才会存储梯度的值。

In [3]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [4]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

在这段代码中，我们逐步进行了自动微分计算，最终得到 `x.grad`。

1. `x = torch.arange(4.0)`：创建了一个包含 `[0., 1., 2., 3.]` 的张量。
2. `x.requires_grad_(True)`：将 `x` 设置为需要计算梯度。
3. `y = 2 * torch.dot(x, x)`：计算 `y`，其中 `torch.dot(x, x)` 是 \(x\) 和自身的点积，结果是 \(0^2 + 1^2 + 2^2 + 3^2 = 14\)。因此 \(y = 2 \times 14 = 28\)。
4. `y.backward()`：对 `y` 进行反向传播，计算 \(x\) 的梯度。
5. `x.grad`：得到 \(y\) 对 \(x\) 的梯度。由于 \(y = 2 \sum x_i^2\)，所以梯度为 \(\frac{\partial y}{\partial x_i} = 4x_i\)，最终结果是 `[0., 4., 8., 12.]`。

In [5]:
x.grad == 4 * x

tensor([True, True, True, True])

In [6]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

In [7]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

In [8]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [9]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

In [10]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [11]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

In [12]:
a.grad == d / a

tensor(True)